**Model Run For Extreme value**

In this file, the model run as well as sensitivity analysis are conducted:

1. Initial Model Run
2. Extreme Value Test

In [1]:
# import necessary libraries and model
from model import AdaptationModel
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict 
import pandas as pd
from IPython.display import display, HTML

In [2]:
# set-up model parameters
# run time: 100 years --> 400 quarters, so 400 ticks in total
run_length = 400
# set the number of replications 
replication_number = 5
# set the number of households
number_of_households = 500

**Initial Run**

In [3]:
# use defaultdict to store the results with different seeds
model_results =  defaultdict(list)
agent_results = defaultdict(list)
# Run the model based on the parameters above, no subsidy included
for i in range(replication_number):
    seed = i  # set the seed
    model = AdaptationModel(number_of_households= number_of_households,subsidy_rate= 0.5, income_threshold=4000, saving_threshold = 0.5, harvey_probability = 0.07, flood_map_choice="harvey", network="no_network", seed=seed)
    for tick in range(run_length):
        model.step() 
    model_data = model.datacollector.get_model_vars_dataframe()
    agent_data = model.datacollector.get_agent_vars_dataframe()
    model_results[i].append(model_data)
    agent_results[i].append(agent_data)
        

**Save the Results**

In [4]:
def convert_to_dataframe(results, replication_number):
    """Convert the results to a dataframe, single index"""
    dfs = [pd.concat(results[i], keys=[i], names=['replication/seed']) for i in range(replication_number)]
    dataframe = pd.concat(dfs)
    dataframe.reset_index(inplace=True)
    dataframe.rename(columns={'level_1': 'Step'}, inplace=True)
    return dataframe

In [5]:
# convert the model results to dataframe
model_dataframe = convert_to_dataframe(model_results, replication_number)

In [6]:
# Save the results to csv files
model_dataframe.to_csv(f"../result_extremevalue/{'model_results_initial'}.csv", index=False)

**Extreme Value Test**

In [7]:
# Parameters for the sensitivity analysis
subsidy_rate = [0, 0.25, 0.5, 0.75, 1]   # subsidy rate offered
income_threshold = [0, 2000, 4000, 8000, 12000] # agents which have lower income then the threshold eligible for subsidy
saving_threshold = [0, 0.25, 0.5, 0.75, 1] # agent calculate_saving(self) threshold, for saving or consuming
flood_probabilities = [0.01, 0.03, 0.07, 0.3, 0.7, 1] # different flood probabilities for harvey map

**1. Saving Threshold Extreme Value**

In [8]:
# use defaultdict to store the results with different seeds
model_results_saving_threshold =  defaultdict(list)
agent_results_saving_threshold = defaultdict(list)
# Run the sensitivity for different saving thresholds, no subsidy included
for rate in saving_threshold:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate= 0.5, income_threshold=4000, saving_threshold = rate, harvey_probability = 0.07, flood_map_choice="harvey", network="no_network", seed=seed)
        for tick in range(run_length):
            model.step() 
        model_data2 = model.datacollector.get_model_vars_dataframe()
        agent_data2 = model.datacollector.get_agent_vars_dataframe()
        agent_data2.reset_index(inplace=True)
        model_results_saving_threshold[i,rate].append(model_data2)
        agent_results_saving_threshold[i,rate].append(agent_data2)


**Save the Results**

In [9]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in model_results_saving_threshold.items():
    i, rate = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_saving_threshold = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'rate', 'Step'])

# Reset the index 
model_dataframe_saving_threshold.reset_index(inplace=True)


In [10]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in agent_results_saving_threshold.items():
    i, rate = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_saving_threshold = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'rate', 'index'])

# Reset the index 
agent_dataframe_saving_threshold.reset_index(inplace=True)
# drop index column
agent_dataframe_saving_threshold.drop(columns=['index'], inplace=True)


In [11]:
# Save the results to csv files
model_dataframe_saving_threshold.to_csv(f"../result_extremevalue/{'model_sensitivity_results_saving_threshold'}.csv", index=False)
# agent_dataframe_saving_threshold.to_csv(f"../result_extremevalue/{'agent_sensitivity_results_saving_threshold'}.csv", index=False)

In [12]:
#upload the data with the following code
# change the file name to the file you want to load
#model_dataframe_saving_threshold = pd.read_csv(f"../result_extremevalue/{'model_sensitivity_results_saving_threshold'}.csv")

**2.Income Threshold**

In [13]:
# use defaultdict to store the results with different seeds
model_results_income_threshold =  defaultdict(list)
agent_results_income_threshold = defaultdict(list)
# Run the sensitivity for different income thresholds, when subsidy is 10% and saving threshold 0.25
for income in income_threshold:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate= 0.5, income_threshold=income, saving_threshold = 0.5, harvey_probability = 0.07, flood_map_choice="harvey", network="no_network", seed=seed)
        for tick in range(run_length):
            model.step() 
        model_data3 = model.datacollector.get_model_vars_dataframe()
        agent_data3 = model.datacollector.get_agent_vars_dataframe()
        agent_data3.reset_index(inplace=True)
        model_results_income_threshold[i,income].append(model_data3)
        agent_results_income_threshold[i,income].append(agent_data3)

**Save the Results**

In [14]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each key
for key, result_list in model_results_income_threshold.items():
    i, income = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_income_threshold = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'income_threshold', 'Step'])

# Reset the index 
model_dataframe_income_threshold.reset_index(inplace=True)



In [15]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in agent_results_income_threshold.items():
    i, income = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_income_threshold = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'income_threshold', 'index'])

# Reset the index 
agent_dataframe_income_threshold.reset_index(inplace=True)
# drop index column
agent_dataframe_income_threshold.drop(columns=['index'], inplace=True)


In [16]:
# save the results to csv files
model_dataframe_income_threshold.to_csv(f"../result_extremevalue/{'model_sensitivity_results_income_threshold'}.csv", index=False)
# agent_dataframe_income_threshold.to_csv(f"../result_extremevalue/{'agent_sensitivity_results_income_threshold'}.csv", index=False)

In [17]:
# upload the data with the following code
# change the file name to the file you want to load
# model_dataframe_income_threshold = pd.read_csv(f"../result_extremevalue/{'model_sensitivity_results_income_threshold'}.csv")

**3.Subsidy Rate**

In [18]:
# use defaultdict to store the results with different seeds
model_results_subsidy_rate =  defaultdict(list)
agent_results_subsidy_rate = defaultdict(list)
# Run the sensitivity for different income thresholds, when income is 3000 and saving threshold 0.25
for subsidy in subsidy_rate:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate= subsidy, income_threshold=4000, saving_threshold = 0.5, harvey_probability = 0.07, flood_map_choice="harvey", network="no_network", seed=seed)
        for tick in range(run_length):
            model.step() 
        model_data4 = model.datacollector.get_model_vars_dataframe()
        agent_data4 = model.datacollector.get_agent_vars_dataframe()
        agent_data4.reset_index(inplace=True)
        model_results_subsidy_rate[i,subsidy].append(model_data4)
        agent_results_subsidy_rate[i,subsidy].append(agent_data4)

**Save the Results**

In [19]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each key
for key, result_list in model_results_subsidy_rate.items():
    i, subsidy = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_subsidy_rate = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'subsidy_rate', 'Step'])

# Reset the index 
model_dataframe_subsidy_rate.reset_index(inplace=True)



In [20]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each key
for key, result_list in agent_results_subsidy_rate.items():
    i, subsidy = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_subsidy_rate = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'subsidy_rate', 'index'])

# Reset the index 
agent_dataframe_subsidy_rate.reset_index(inplace=True)
# drop index column
agent_dataframe_subsidy_rate.drop(columns=['index'], inplace=True)


In [21]:
# save the results to csv files
model_dataframe_subsidy_rate.to_csv(f"../result_extremevalue/{'model_sensitivity_results_subsidy_rate'}.csv", index=False)
# agent_dataframe_subsidy_rate.to_csv(f"../result_extremevalue/{'agent_sensitivity_results_subsidy_rate'}.csv", index=False)

In [22]:
# upload the data with the following code
# change the file name to the file you want to load
# model_dataframe_subsidy_rate = pd.read_csv(f"../result_extremevalue/{'model_sensitivity_results_subsidy_rate'}.csv")

**4. Flood Probabilities**

In [23]:
# use defaultdict to store the results with different seeds
model_results_flood_prob =  defaultdict(list)
agent_results_flood_prob = defaultdict(list)
# Run the sensitivity for different flood prob, no subsidy included
for prob in flood_probabilities:
    for i in range(replication_number):
        seed = i  # set the seed
        model = AdaptationModel(number_of_households= number_of_households,subsidy_rate= 0.5, income_threshold=4000, saving_threshold = 0.5, harvey_probability = prob, flood_map_choice="harvey", network="no_network", seed=seed)
        
        for tick in range(run_length):
            model.step() 
        model_data5 = model.datacollector.get_model_vars_dataframe()
        agent_data5 = model.datacollector.get_agent_vars_dataframe()
        agent_data5.reset_index(inplace=True)
        model_results_flood_prob[i,prob].append(model_data5)
        agent_results_flood_prob[i,prob].append(agent_data5)

**Save the Results**

In [24]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in model_results_flood_prob.items():
    i, prob = key
    concatenated_dfs[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
model_dataframe_flood_prob = pd.concat(concatenated_dfs.values(), keys=concatenated_dfs.keys(), names=['replication/seed', 'flood_probability', 'Step'])

# Reset the index 
model_dataframe_flood_prob.reset_index(inplace=True)



In [25]:
# Create a dictionary to hold the concatenated dataframes
concatenated_dfs2 = {}

# Concatenate the dataframes for each (i, rate) key
for key, result_list in agent_results_flood_prob.items():
    i, prob = key
    concatenated_dfs2[key] = pd.concat(result_list)

# Convert the dictionary of dataframes to a single dataframe
agent_dataframe_flood_prob = pd.concat(concatenated_dfs2.values(), keys=concatenated_dfs2.keys(), names=['replication/seed', 'flood_probability', 'index'])

# Reset the index 
agent_dataframe_flood_prob.reset_index(inplace=True)
# drop index column
agent_dataframe_flood_prob.drop(columns=['index'], inplace=True)


In [26]:
# save the results to csv files
model_dataframe_flood_prob.to_csv(f"../result_extremevalue/{'model_sensitivity_results_flood_prob'}.csv", index=False)
# agent_dataframe_flood_prob.to_csv(f"../result_extremevalue/{'agent_sensitivity_results_flood_prob'}.csv", index=False)